In [10]:
import os
os.environ["PYTHONNET_RUNTIME"] = "coreclr"
os.environ["DOTNET_SYSTEM_DRAWING_USE_GDIPLUS"] = "1"

import clr
import numpy as np
import time
from datetime import timedelta
from pythonnet import load
from System import Environment, Array, String

# Load .NET Core runtime
load("coreclr")

# Try to import System.IO; fallback to os.chdir if it fails
try:
    from System.IO import Directory, Path, File
    use_dotnet_dir = True
except Exception as e:
    print(f"Note: System.IO import failed ({e}), using os.chdir instead")
    import os
    use_dotnet_dir = False

# Define DWSIM path
dwsimpath = "/usr/local/lib/dwsim/"

# Set working directory
if use_dotnet_dir:
    Directory.SetCurrentDirectory(dwsimpath)
else:
    os.chdir(dwsimpath)

# Add DWSIM assemblies
clr.AddReference(dwsimpath + "CapeOpen.dll")
clr.AddReference(dwsimpath + "DWSIM.Automation.dll")
clr.AddReference(dwsimpath + "DWSIM.Interfaces.dll")
clr.AddReference(dwsimpath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwsimpath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwsimpath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwsimpath + "DWSIM.Inspector.dll")
clr.AddReference(dwsimpath + "System.Buffers.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.ThermoC.dll")

# Now import DWSIM types
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings

print("DWSIM imports successful!")

DWSIM imports successful!


In [11]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [12]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [13]:
# Add Compounds
cnames = ["Water"]
sim.AddCompound("Water")

In [14]:
# Adding objects
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
four = sim.AddFlowsheetObject('Material Stream','4')
five = sim.AddFlowsheetObject('Material Stream','5')
six = sim.AddFlowsheetObject('Material Stream','6')
seven = sim.AddFlowsheetObject('Material Stream','7')
eight = sim.AddFlowsheetObject('Material Stream','8')
nine = sim.AddFlowsheetObject('Material Stream','9')
MIX_1 = sim.AddFlowsheetObject('Stream Mixer','MIX-1')
MIX_2 = sim.AddFlowsheetObject('Stream Mixer','MIX-2')
MIX_3 = sim.AddFlowsheetObject('Stream Mixer','MIX-3')

In [15]:
# Calling all generated objects
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
four = four.GetAsObject()
five = five.GetAsObject()
six = six.GetAsObject()
seven = seven.GetAsObject()
eight = eight.GetAsObject()
nine = nine.GetAsObject()
MIX_1 = MIX_1.GetAsObject()
MIX_2 = MIX_2.GetAsObject()
MIX_3 = MIX_3.GetAsObject()

In [16]:
# Connectting objects
sim.ConnectObjects(one.GraphicObject , MIX_1.GraphicObject, -1, -1)
sim.ConnectObjects(two.GraphicObject , MIX_1.GraphicObject, -1, -1)
sim.ConnectObjects(MIX_1.GraphicObject , three.GraphicObject, -1, -1)
sim.ConnectObjects(four.GraphicObject , MIX_2.GraphicObject, -1, -1)
sim.ConnectObjects(five.GraphicObject , MIX_2.GraphicObject, -1, -1)
sim.ConnectObjects(MIX_2.GraphicObject , six.GraphicObject, -1, -1)
sim.ConnectObjects(seven.GraphicObject , MIX_3.GraphicObject, -1, -1)
sim.ConnectObjects(eight.GraphicObject , MIX_3.GraphicObject, -1, -1)
sim.ConnectObjects(MIX_3.GraphicObject , nine.GraphicObject, -1, -1)

In [17]:
# layout for aesthetics
sim.NaturalLayout()

In [18]:
# Setting parameters
one.SetOverallComposition(Array[float]([1]))
one.SetTemperature(350.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

two.SetOverallComposition(Array[float]([1]))
two.SetTemperature(450.0) # K
two.SetPressure(501325.0) # Pa
two.SetMassFlow(1.0) # kg/s

four.SetOverallComposition(Array[float]([1]))
four.SetTemperature(350.0) # K
four.SetPressure(101325.0) # Pa
four.SetMassFlow(1.0) # kg/s

five.SetOverallComposition(Array[float]([1]))
five.SetTemperature(450.0) # K
five.SetPressure(501325.0) # Pa
five.SetMassFlow(1.0) # kg/s

seven.SetOverallComposition(Array[float]([1]))
seven.SetTemperature(350.0) # K
seven.SetPressure(101325.0) # Pa
seven.SetMassFlow(1.0) # kg/s

eight.SetOverallComposition(Array[float]([1]))
eight.SetTemperature(450.0) # K
eight.SetPressure(501325.0) # Pa
eight.SetMassFlow(1.0) # kg/s

'8: mass flow set to 1 kg/s'

In [19]:
# property package
Steam_table = sim.CreateAndAddPropertyPackage("Steam Tables (IAPWS-IF97)")

In [20]:
# Calc Modes
MIX_1.PressureCalculation = MIX_1.PressureCalculation.Minimum
MIX_2.PressureCalculation = MIX_2.PressureCalculation.Average
MIX_3.PressureCalculation = MIX_3.PressureCalculation.Maximum

In [21]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [23]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), "/workspace/00 FlowSheet Automation/00 Mixer Automation/00 Mixer.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)